In [2]:
from flickrapi import FlickrAPI

In [4]:
key = '0e4e7a8ceb3f350abd0a743812365c51'
secret = '3172a3f6ec67c50f'
sizes = ['url_c', 'url_z', 'url_m', 'url_n', 'url_s', 'url_t']

In [7]:
def get_photos(image_tag):
    extras = ','.join(sizes)
    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,  # it will search by image title and image tags
                            extras=extras,  # get the urls for each size we want
                            privacy_filter=1,  # search only for public photos
                            per_page=50,
                            sort='relevance')  # we want what we are looking for to appear first
    return photos

In [16]:
get_photos('croquetas')

In [15]:
def get_url(photo):
    for i in range(len(sizes)):  # makes sure the loop is done in the order we want
        url = photo.get(sizes[i])
        if url:  # if url is None try with the next size
            return url

In [5]:
def get_urls(image_tag, max):
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            url = get_url(photo)  # get preffered size url
            if url:
                urls.append(url)
                counter += 1
            # if no url for the desired sizes then try with the next photo
        else:
            break

    return urls

In [8]:
!pwd

/home/dsc/TFM_food_recognition


In [10]:
from progress.bar import Bar
import requests
import os
import sys
import time

def create_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [20]:
def download_images(urls, path):
    create_folder(path)  # makes sure path exists

    for url in urls:
        image_name = url.split("/")[-1]
        image_path = os.path.join(path, image_name)

        if not os.path.isfile(image_path):  # ignore if already downloaded
            response=requests.get(url,stream=True)

            with open(image_path,'wb') as outfile:
                outfile.write(response.content)

In [22]:
from flickr import get_urls
from downloader import download_images
import os
import time

all_species = ['blue jay', 'northern cardinal', 'american goldfinch']
images_per_species = 10

def download():
    for species in all_species:

        print('Getting urls for', specie)
        urls = get_urls(specie, images_per_specie)
        
        print('Downloading images for', specie)
        path = os.path.join('data', specie)

        download_images(urls, path)


ImportError: cannot import name 'get_urls' from 'flickrapi' (/home/dsc/anaconda3/lib/python3.7/site-packages/flickrapi/__init__.py)

In [ ]:
start_time = time.time()

download()

print('Took', round(time.time() - start_time, 2), 'seconds')